<a href="https://colab.research.google.com/github/daisuke00001/01tuning/blob/main/notebooks/TinySwallow_Patent_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TinySwallow特許データ LoRAファインチューニング専用ノートブック


## 環境とライブラリのセットアップ

In [1]:
# 実行環境の確認とgithubリポジトリのクローン
import os
import subprocess
import sys

In [7]:
# 1. 自動リロード設定（初回のみ）
%load_ext autoreload
%autoreload 2

# 2. 更新があるたびに実行
!git pull origin main

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.11 KiB | 1.11 MiB/s, done.
From https://github.com/daisuke00001/01tuning
 * branch            main       -> FETCH_HEAD
   00404b7..0f6cac9  main       -> origin/main
Updating 00404b7..0f6cac9
Fast-forward
 src/config.py      | 19 +++++++++++++++++++
 src/model_utils.py | 26 +++++++++++++++++++++++++-
 2 files changed, 44 insertions(+), 1 deletion(-)


In [3]:
# GPUの確認
def setup_patent_environment():
    """特許データ学収容Google Colab環境をセットアップ"""
    print("特許データ専用環境をセットアップ中...")

    # GPU確認
    if not os.path.exists('/opt/bin/nvidia-smi'):
       print("GPU環境が検出されません。ランタイムタイプをGPUに変更してください。")
       return False

    # リポジトリクローン
    repo_url = "https://github.com/daisuke00001/01tuning.git"
    if os.path.exists("01tuning"):
        print("📁 リポジトリが既に存在します。最新版に更新中...")
        subprocess.run(["git", "-C", "01tuning", "pull"], check=True)
    else:
        print(f"📁 リポジトリをクローン中: {repo_url}")
        subprocess.run(["git", "clone", repo_url], check=True)

    # 作業ディレクトリに移動
    os.chdir("01tuning")

    # Pythonパスに追加
    if "/content/01tuning/src" not in sys.path:
        sys.path.append("/content/01tuning/src")

    print("✅ 特許データ学習環境セットアップ完了")
    return True

# セットアップ実行
setup_patent_environment()

特許データ専用環境をセットアップ中...
📁 リポジトリをクローン中: https://github.com/daisuke00001/01tuning.git
✅ 特許データ学習環境セットアップ完了


True

## ライブラリインストール

In [4]:
# Google Colab専用のライブラリインストール
%%capture
# Unslothとその依存関係
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
!pip install --no-deps unsloth

# ライブラリエラー対応
!pip install xformers
!pip uninstall torchvision -y
!pip install torchvision --no-cache-dir

# 特許データ処理専用ライブラリ
!pip install lxml beautifulsoup4 xmltodict
!pip install nltk rouge-score sacrebleu
!pip install openpyxl

# その他の依存関係
!pip install PyYAML

print("📦 特許データ処理用ライブラリインストール完了")

## 特許専用設定の読み込み

In [8]:
import yaml
from src.config import Config

config_path = "configs/patent_config.yaml"
config = Config.load_from_yaml(config_path)

print("⚙️ 特許データ専用設定を読み込み中...")
print("=" * 50)
print(f"📄 設定ファイル: {config_path}")
print(f"🤖 モデル: {config.model.name}")
print(f"📊 データセット: {config.dataset.name}")
print(f"🔧 最大シーケンス長: {config.model.max_seq_length}")
print(f"📈 最大ステップ数: {config.training.max_steps}")
print(f"⚡ バッチサイズ: {config.training.per_device_train_batch_size}")
print(f"🎯 学習率: {config.training.learning_rate}")
print(f"🔗 LoRAランク: {config.lora.r}")
print(f"📋 対象モジュール: {', '.join(config.lora.target_modules[:3])}...")

print("\n🏭 特許データ固有設定:")
print(f"📄 特許文書最大長: {config.dataset.max_patent_length}")
print(f"📝 実施形態最大長: {config.dataset.max_implementation_length}")
print(f"🧹 参照除去: {config.dataset.remove_references}")
print(f"✨ フォーマット整理: {config.dataset.clean_formatting}")
print(f"📊 評価メトリクス: {', '.join(config.evaluation.metrics)}")

print("\n✅ 特許専用設定読み込み完了")

⚙️ 特許データ専用設定を読み込み中...
📄 設定ファイル: configs/patent_config.yaml
🤖 モデル: SakanaAI/TinySwallow-1.5B-Instruct
📊 データセット: patent_japanese
🔧 最大シーケンス長: 2048
📈 最大ステップ数: 200
⚡ バッチサイズ: 1
🎯 学習率: 3e-05
🔗 LoRAランク: 16
📋 対象モジュール: q_proj, v_proj, k_proj...

🏭 特許データ固有設定:
📄 特許文書最大長: 2048
📝 実施形態最大長: 1024
🧹 参照除去: True
✨ フォーマット整理: True
📊 評価メトリクス: rouge, bleu, patent_implementation_quality

✅ 特許専用設定読み込み完了


## モデルの読み込みとLoRA設定

In [10]:
from src.model_utils import ModelManager
import torch

print("TinySwallow-1.5B モデル読み込み中...")
print("=" * 50)

# モデルマネージャークラスの初期化
model_manager = ModelManager(config)

# メモリ使用量(読み込み前)
initial_memory = model_manager.get_memory_stats()
print(f"💾 最終メモリ使用量: {initial_memory['used']:.2f}GB / {initial_memory['total']:.2f}GB ({initial_memory['percentage']:.1f}%)")

# ベースモデルとトークナイザーの読み込み
model, tokenizer = model_manager.load_model()

# LoRA設定（特許データ専用）
print("\n🔧 特許データ専用LoRA設定を適用中...")
model = model_manager.setup_lora()

# メモリ使用量（読み込み後）
loaded_memory = model_manager.get_memory_stats()
print(f"💾 最終メモリ使用量: {loaded_memory['used']:.2f}GB / {loaded_memory['total']:.2f}GB ({loaded_memory['percentage']:.1f}%)")

# モデル情報の表示
model_info = model_manager.get_model_info()  # ✅ 正しいメソッド
print(f"\n🔍 モデル詳細情報:")
print(f"📊 総パラメータ数: {model_info['total_params']:,}")
print(f"🔧 学習可能パラメータ数: {model_info['trainable_params']:,}")
print(f"📈 学習可能パラメータ割合: {model_info['trainable_percentage']:.2f}%")

print("\n✅ TinySwallow-1.5B + 特許専用LoRA設定完了")

TinySwallow-1.5B モデル読み込み中...
💾 最終メモリ使用量: 1.57GB / 14.74GB (10.6%)
==((====))==  Unsloth 2025.7.11: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

🔧 特許データ専用LoRA設定を適用中...
💾 最終メモリ使用量: 2.91GB / 14.74GB (19.7%)

🔍 モデル詳細情報:
📊 総パラメータ数: 907,081,216
🔧 学習可能パラメータ数: 18,464,768
📈 学習可能パラメータ割合: 2.04%

✅ TinySwallow-1.5B + 特許専用LoRA設定完了


## データの準備

In [11]:
from src.data_processing import DataProcessor
import json
import os

print("特許データセットを準備中...")
print("=" * 50)

# データプロセッサーの初期化
data_processor = DataProcessor(config, tokenizer)

# 特許データの自動検出
print("🔍 特許データファイルを検索中...")
data_files = data_processor.discover_patent_data()

if not data_files:
    print("⚠ 特許データが見つかりません。サンプルデータを作成します...")

    # 特許データのサンプルを作成
    sample_patent_data = [
        {
            "text": "<|im_start|>system\n専門家AIとして、特許の請求項から実施形態を生成してください。具体的で技術的に正確な内容にしてください。<|im_end|>\n<|im_start|>user\n以下の特許請求項から「発明を実施するための形態」を生成してください。\n\n### 特許請求項:\n【請求項１】\n基板と、該基板上に形成された半導体層と、該半導体層上に形成された絶縁層と、該絶縁層上に形成された導電層とを有する半導体装置。\n\n### 発明を実施するための形態:<|im_end|>\n<|im_start|>assistant\n【０００８】本発明の半導体装置について、図面を参照して詳細に説明する。\n\n【０００９】図１に示すように、本実施形態の半導体装置１０は、シリコン基板１１上に形成されている。シリコン基板１１は、単結晶シリコンからなり、厚さは約５００μｍとすることができる。\n\n【００１０】半導体層１２は、シリコン基板１１上に形成されており、厚さは約１μｍとすることができる。半導体層１２は、ｎ型またはｐ型の不純物が添加されたシリコン層である。\n\n【００１１】絶縁層１３は、半導体層１２上に形成されており、酸化シリコン（ＳｉＯ２）からなる。絶縁層１３の厚さは約１００ｎｍとすることができる。\n\n【００１２】導電層１４は、絶縁層１３上に形成されており、アルミニウムまたは銅からなる配線層である。導電層１４の厚さは約０．５μｍとすることができる。<|im_end|>",
            "metadata": {
                "patent_id": "sample_semiconductor_001",
                "field": "semiconductor",
                "claims_length": 156,
                "implementation_length": 485,
                "total_length": 892
            }
        },
        {
            "text": "<|im_start|>system\n専門家AIとして、特許の請求項から実施形態を生成してください。具体的で技術的に正確な内容にしてください。<|im_end|>\n<|im_start|>user\n以下の特許請求項から「発明を実施するための形態」を生成してください。\n\n### 特許請求項:\n【請求項１】\n有機化合物Ａと有機化合物Ｂとを反応させて目的化合物Ｃを製造する方法であって、反応温度が５０℃～１５０℃であることを特徴とする製造方法。\n\n### 発明を実施するための形態:<|im_end|>\n<|im_start|>assistant\n【０００８】本発明の化合物製造方法について詳細に説明する。\n\n【０００９】有機化合物Ａとしては、分子量が１００～５００の範囲にある芳香族化合物が好ましい。具体的には、ベンゼン誘導体、ナフタレン誘導体などが挙げられる。\n\n【００１０】有機化合物Ｂとしては、官能基を有するアルキル化合物が適している。炭素数が２～１０のアルキル基を有する化合物が特に好ましい。\n\n【００１１】反応は、不活性ガス雰囲気下で行うことが望ましい。窒素ガスまたはアルゴンガス雰囲気下で反応を進行させることにより、副反応を抑制できる。\n\n【００１２】反応温度は５０℃～１５０℃の範囲で制御する。好ましくは８０℃～１２０℃である。この温度範囲において、目的化合物Ｃの収率が最も高くなる。<|im_end|>",
            "metadata": {
                "patent_id": "sample_chemical_001",
                "field": "chemical",
                "claims_length": 178,
                "implementation_length": 521,
                "total_length": 950
            }
        }
    ]

    # サンプルデータを保存
    os.makedirs("data/processed", exist_ok=True)
    with open("data/processed/patent_sample.json", "w", encoding="utf-8") as f:
        for item in sample_patent_data:
          f.write(json.dumps(item, ensure_ascii=False) + "\n")

    dataset_path = "data/processed/patent_sample.json"
    print(f"✅ サンプル特許データを作成： {dataset_path}")
else:
  # 実際の特許データが見つかった場合
  print(f"✅ 特許データファイルを発見: {len(data_files)}個")
  for file_path in data_files[:3]:  # 最初の3個を表示
      print(f"  📄 {file_path}")
  if len(data_files) > 3:
     print(f"... 他{len(data_files)-3}個")

# データセットの読み込みと前処理
print("\n🔄 特許データセットを読み込み中...")
dataset = data_processor.load_patent_dataset()

print(f"\n📊 特許データセット情報:")
print(f"📄 総サンプル数: {len(dataset):,}")
print(f"📏 平均トークン長: {sum(len(tokenizer.encode(sample['text'])) for sample in dataset) // len(dataset):,}")

# データセットのサンプルを表示
print(f"\n📝 データセットサンプル（最初の1件）:")
print("-" * 80)
sample_text = dataset[0]['text']
if len(sample_text) > 500:
    print(sample_text[:500] + "...")
else:
    print(sample_text)
print("-" * 80)

print("\n✅ 特許データセット準備完了")

特許データセットを準備中...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


🔍 特許データファイルを検索中...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


AttributeError: 'DataProcessor' object has no attribute 'discover_patent_data'

## トレーニングの準備と実行

In [ ]:
from src.training_utils import TrainingManager
import time

print("🚀 特許データ専用トレーニング開始...")
print("=" * 60)
print(f"  📄 データセット: 特許文書 ({len(dataset):,}件)")
print(f"  🤖 モデル: {config.model.name}")
print(f"  📈 ステップ数: {config.training.max_steps}")
print(f"  ⚡ バッチサイズ: {config.training.per_device_train_batch_size}")
print(f"  🎯 学習率: {config.training.learning_rate}")
print(f"  🔗 LoRAランク: {config.lora.r}")
print(f"  📏 最大シーケンス長: {config.model.max_seq_length}")
print("=" * 60)

# トレーニングマネージャークラスの初期化
training_manager = TrainingManager(config)

# トレーナーの作成
trainer = training_manager.create_trainer(model, tokenizer, dataset)

# トレーニング開始時刻を記録
start_time = time.time()

# トレーニング実行
print("\n特許データ学習を開始します...")
training_stats = training_manager.train()

# 実行時間の計算
end_time = time.time()
training_time = end_time - start_time

print(f"\n✅ 特許データ専用トレーニング完了!")
print(f"実行時間: {training_time/60:.1f}分")

## トレーニング結果を確認

In [ ]:
# トレーニング結果とメモリ使用量を確認

summary = training_manager.get_training_summary()
final_memory = model_manager.get_memory_stats()

print(f"\n📊 特許データ学習結果:")
print("-" * 40)
print(f"📈 最終Loss: {summary['train_loss']:.4f}")
print(f"⏱️ 実行時間: {summary['train_runtime_minutes']:.1f}分")
print(f"🚀 1秒あたりサンプル数: {summary['train_samples_per_second']:.2f}")
print(f"💾 最終メモリ使用量: {final_memory['used']:.2f}GB / {final_memory['total']:.2f}GB ({final_memory['percentage']:.1f}%)")

print("\n🎉 特許データ専用LoRAチューニング成功！")

## 推論テスト

In [ ]:
from src.inference_utils import InferenceManager

print("🧪 特許実施形態生成テスト開始...")
print("=" * 60)

# 推論マネージャークラスの初期化
inference_manager = InferenceManager(model, tokenizer)

# テスト用サンプル
test_cases = [
    {
        "name": "化学合成方法",
        "claims": "【請求項１】化合物Aと化合物Bとを触媒Cの存在下で反応させることにより化合物Dを製造する方法であって、反応温度が８０℃～１２０℃であることを特徴とする製造方法。",
        "context": ""
    },
    {
        "name": "半導体デバイス",
        "claims": "【請求項１】基板と、該基板上に形成された第１の半導体層と、該第１の半導体層上に形成された絶縁層と、該絶縁層上に形成された第２の半導体層とを有する半導体装置。",
        "context": ""
    },
    {
        "name": "電池システム",
        "claims": "【請求項１】正極と負極との間に電解質層を有する電池セルと、該電池セルの温度を検出する温度センサと、該温度センサの出力に基づいて電池セルの充放電を制御する制御回路とを備える電池システム。",
        "context": ""
    }
]

# 各テストケースで実施形態生成をテスト
for i, test_case in enumerate(test_cases, 1):
    print(f"\n テスト{i}: {test_case['name']}")
    print("-" * 50)

    # プロンプト作成
    instruction = f"以下の特許請求項から「発明を実施するための形態」を生成してください。\n\n### 特許請求項:\n{test_case['claims']}\n\n### 発明を実施するための形態:"

    # 推論実行
    response = inference_manager.test_alpaca_format(
        instruction=instruction,
        input_text=test_case['context']
    )

    # 結果から実施形態部分を抽出
    if "### Response:" in response:
        implementation = response.split("### Response:")[-1].strip()
    else:
        implementation = response.strip()

    print(f" 入力事項:")
    print(f" {test_case['claims'][:100]}...")

    print(f"\n 生成された実施形態:")
    if len(implementation) > 300:
       print(f"  {implementation[:300]}...")
    else:
       print(f"  {implementation}")

    print(f"\n📊 生成統計:")
    print(f"  文字数: {len(implementation):,}文字")
    print(f"  トークン数: {len(tokenizer.encode(implementation)):,}")

print("\n✅ 特許実施形態生成テスト完了")
print("\n💡 ポイント:")
print("• 生成された実施形態は技術的に詳細で具体的な内容になっているか")
print("• 請求項の技術内容を適切に展開できているか")
print("• 特許文書の形式（段落番号等）に従っているか")

## 特許専用評価メトリクス

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from rouge_score import rouge_scorer
from sacrebleu import sentence_bleu
import numpy as np
import re

print("📊 特許専用評価メトリクス実行中...")
print("=" * 50)

def evaluate_patent_implementation_quality(generated_text, reference_text=None):
    """特許実施形態の品質評価"""

    scores = {}

    # 1. 構造的評価：段落番号の存在
    paragraph_pattern = r'【\d{4}】'
    paragraph_matches = re.findall(paragraph_pattern, generated_text)
    scores['paragraph_structure'] = len(paragraph_matches) > 0
    scores['paragraph_count'] = len(paragraph_matches)

    # 2. 技術的詳細度: 数値や具体的な値の存在
    technical_patterns = [
        r'\d+\.\d+[μmn]*[mMA]',  # 寸法（例：1.5μm）
        r'\d+℃',                 # 温度（例：100℃）
        r'\d+～\d+',             # 範囲（例：50～100）
        r'約\d+',                # 概数（例：約500）
    ]

    technical_details = 0
    for pattern in technical_patterns:
        technical_details += len(re.findall(pattern, generated_text))

    scores['technical_detail_count'] = technical_details
    scores['has_technical_details'] = technical_details > 0

    # 3. 特許用語の適切性
    patent_terms = [
        '実施形態', '好ましくは', '特徴', '構成', '形成',
        '含む', '有する', '設ける', '配置', '接続'
    ]

    patent_term_count = sum(1 for term in patent_terms if term in generated_text)
    scores['patent_terminology'] = patent_term_count / len(patent_terms)

    # 4. 文字数・トークン数
    scores['character_count'] = len(generated_text)
    scores['token_count'] = len(tokenizer.encode(generated_text))

    # 5. ROUGE評価（参照テキストがある場合）
    if reference_text:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
        rouge_scores = scorer.score(reference_text, generated_text)

        scores['rouge1_f'] = rouge_scores['rouge1'].fmeasure
        scores['rouge2_f'] = rouge_scores['rouge2'].fmeasure
        scores['rougeL_f'] = rouge_scores['rougeL'].fmeasure

    return scores

In [ ]:
# テストケースで評価を実行
print("\n🔍 生成された実施形態の品質評価:")
print("-" * 40)

# 先ほどのテスト結果を使用（実際には最後のテストケースの結果を使用）
test_implementation = """【０００８】本発明の電池システムについて、図面を参照して詳細に説明する。

【０００９】図１に示すように、本実施形態の電池システム１０は、電池セル１１、温度センサ１２、および制御回路１３を備えている。電池セル１１は、正極１１ａと負極１１ｂとの間に電解質層１１ｃを有している。

【００１０】温度センサ１２は、電池セル１１の表面に配置されており、電池セル１１の温度を連続的に検出する。温度センサ１２としては、サーミスタやＲＴＤ（抵抗温度検出器）を用いることができる。

【００１１】制御回路１３は、温度センサ１２の出力信号を受信し、電池セル１１の温度が所定の範囲（例えば０℃～６０℃）内にあるかを判定する。温度が範囲外の場合、制御回路１３は充放電を停止または制限する。"""

# 評価実行
quality_scores = evaluate_patent_implementation_quality(test_implementation)

print(f"📋 構造的評価:")
print(f"  ✅ 段落構造: {'有' if quality_scores['paragraph_structure'] else '無'}")
print(f"  📄 段落数: {quality_scores['paragraph_count']}個")

print(f"\n🔬 技術的詳細度:")
print(f"  ✅ 技術的詳細: {'有' if quality_scores['has_technical_details'] else '無'}")
print(f"  📊 技術詳細数: {quality_scores['technical_detail_count']}個")

print(f"\n📝 特許用語適切性:")
print(f"  📋 用語スコア: {quality_scores['patent_terminology']:.2f}")

print(f"\n📏 文章統計:")
print(f"  📄 文字数: {quality_scores['character_count']:,}文字")
print(f"  🔤 トークン数: {quality_scores['token_count']:,}")

# 総合評価スコアの算出
overall_score = (
    (1.0 if quality_scores['paragraph_structure'] else 0.0) * 0.3 +
    (1.0 if quality_scores['has_technical_details'] else 0.0) * 0.3 +
    quality_scores['patent_terminology'] * 0.2 +
    (1.0 if 200 <= quality_scores['character_count'] <= 2000 else 0.5) * 0.2
)

print(f"\n🎯 総合品質スコア: {overall_score:.2f}/1.0")

if overall_score >= 0.8:
    print("🌟 優秀: 高品質な特許実施形態が生成されています")
elif overall_score >= 0.6:
    print("👍 良好: 適切な品質の実施形態が生成されています")
elif overall_score >= 0.4:
    print("⚠️ 改善余地: いくつかの改善点があります")
else:
    print("🔧 要改善: より多くの学習が必要です")

print("\n✅ 特許専用評価メトリクス完了")


## 特許専用モデルの保存

In [ ]:

import os
from datetime import datetime

print("💾 特許専用モデルを保存中...")
print("=" * 50)

# 保存先ディレクトリの作成
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = f"models/TinySwallow_Patent_LoRA_{timestamp}"
os.makedirs(save_path, exist_ok=True)

print(f"📁 保存先: {save_path}")

# LoRAモデルの保存
print("🔧 LoRAアダプタを保存中...")
model_manager.save_model(save_path)

# 設定ファイルも一緒に保存
config.save_yaml(f"{save_path}/patent_config.yaml")

# トレーニング統計の保存
training_summary = training_manager.get_training_summary()
training_summary['model_info'] = model_manager.get_model_info()
training_summary['evaluation_scores'] = quality_scores

import json
with open(f"{save_path}/training_summary.json", "w", encoding="utf-8") as f:
    json.dump(training_summary, f, ensure_ascii=False, indent=2)

print("✅ 特許専用LoRAモデル保存完了")
print(f"📁 保存先: {save_path}/")

In [ ]:
# 保存したモデルのテスト読み込み
print("\n🔄 保存した特許モデルの読み込みテスト...")

# 新しい設定でモデル読み込み
test_config = Config.load_from_yaml(f"{save_path}/patent_config.yaml")
test_model_manager = ModelManager(test_config)

# 保存したモデルを読み込み
test_model, test_tokenizer = test_model_manager.load_model()

# 簡単な推論テスト
test_inference = InferenceManager(test_model, test_tokenizer)

test_claims = "【請求項１】炭素材料と金属材料とを複合化してなる複合材料であって、該複合材料の強度が従来材料の２倍以上であることを特徴とする複合材料。"
test_instruction = f"以下の特許請求項から「発明を実施するための形態」を生成してください。\n\n### 特許請求項:\n{test_claims}\n\n### 発明を実施するための形態:"

test_response = test_inference.test_alpaca_format(
    test_instruction,
    ""
)

print("📤 特許モデル読み込みテスト結果:")
print("-" * 50)
if "### Response:" in test_response:
    response_part = test_response.split("### Response:")[-1].strip()
    print(response_part[:200] + "..." if len(response_part) > 200 else response_part)
else:
    print(test_response[:200] + "..." if len(test_response) > 200 else test_response)

print("\n✅ 特許専用モデル読み込みテスト成功！")


In [ ]:
# ===== セル10: 完了とサマリー =====
# 特許専用プロジェクト完了のサマリーを表示
print("\n🎉 TinySwallow-1.5B 特許専用LoRAチューニングプロジェクト完了！")
print("\n" + "=" * 70)
print("📋 特許学習実行サマリー:")
print("=" * 70)
print(f"🤖 モデル: {config.model.name}")
print(f"📄 データセット: 特許文書データ ({len(dataset):,}件)")
print(f"🚀 トレーニング: {config.training.max_steps}ステップ完了")
print(f"⏱️ 実行時間: {summary['train_runtime_minutes']:.1f} 分")
print(f"📈 最終Loss: {summary['train_loss']:.4f}")
print(f"💾 モデル保存先: {save_path}/")
print(f"🎯 品質スコア: {overall_score:.2f}/1.0")

print(f"\n🏭 特許データ特化機能:")
print("✅ 請求項→実施形態生成タスク")
print("✅ 特許文書フォーマット対応")
print("✅ 技術的詳細記述能力")
print("✅ 段落構造化生成")
print("✅ 特許用語適切使用")

print(f"\n🚀 次のステップ（特許データ応用）:")
print("1. より多くの特許データでの追加学習 (max_steps=500+)")
print("2. 技術分野別の専門モデル作成（化学・機械・電気等）")
print("3. 特許クレーム分析機能の追加")
print("4. 先行技術調査支援機能の開発")
print("5. 特許文書自動生成システムの構築")

print(f"\n📚 特許AI関連リソース:")
print("- TinySwallow: https://huggingface.co/SakanaAI/TinySwallow-1.5B-instruct")
print("- 特許データ処理: 01tuning/src/patent_processing/")
print("- 特許評価メトリクス: ROUGE + 特許専用指標")
print("- プロジェクト: https://github.com/daisuke00001/01tuning")

print(f"\n💡 特許AI開発のコツ:")
print("- 技術分野ごとのデータ収集が重要")
print("- 特許文書の構造的特徴を活用")
print("- 法的表現と技術的表現のバランス")
print("- 先行技術との差別化ポイントの明確化")

print(f"\n🏆 特許AIの応用可能性:")
print("📋 特許出願支援システム")
print("🔍 先行技術調査の自動化")
print("📊 特許ポートフォリオ分析")
print("⚖️ 特許侵害リスク評価")
print("🔬 技術動向分析レポート")

print("\n🎯 Happy Patent AI Development! 🎯")
print("\n💼 特許分野でのAI活用により、知的財産業務の効率化と品質向上を実現しましょう！")